In [ ]:
!pip install transformers[sentencepiece] spacy rouge

In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.4 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
import torch
import re
import spacy
import pandas as pd
import nltk
import json
import en_core_web_lg
from rouge import Rouge 
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformers import AutoModelForMaskedLM, AutoModel, AutoTokenizer, pipeline

In [ ]:
nlp = en_core_web_lg.load()
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!ls checkpoint-10000

config.json	   scheduler.pt		    trainer_state.json
optimizer.pt	   special_tokens_map.json  training_args.bin
pytorch_model.bin  tokenizer_config.json    vocab.txt
rng_state.pth	   tokenizer.json


In [ ]:
!ls model/bert_med/checkpoint/checkpoint-final

all_results.json   qa.json		    trainer_state.json
config.json	   special_tokens_map.json  training_args.bin
eval_results.json  tokenizer_config.json    train_results.json
pytorch_model.bin  tokenizer.json	    vocab.txt


In [ ]:
question = "Can you give dosage information acetaminophen for children?"

reply = """Use acetaminophen exactly as directed on the label, or as prescribed by your doctor. Do not use in larger or smaller amounts or for longer than recommended.
Do not take more of this medication than is recommended. An overdose of acetaminophen can damage your liver or cause death.
Adults and teenagers who weigh at least 110 pounds (50 kilograms): Do not take more than 1000 milligrams (mg) at one time. Do not take more than 4000 mg in 24 hours.
Children younger than 12 years old: Do not take more than 5 doses of acetaminophen in 24 hours. Use only the number of milligrams per dose that is recommended for the child's weight and age. Use exactly as directed on the label.
Avoid also using other medicines that contain acetaminophen, or you could have a fatal overdose.
If you are treating a child, use a pediatric form of acetaminophen. Use only the special dose-measuring dropper or oral syringe that comes with the specific pediatric form you are using. Carefully follow the dosing directions on the medicine label.
Measure liquid medicinewith the dosing syringe provided, or with a special dose-measuring spoon or medicine cup. If you do not have a dose-measuring device, ask your pharmacist for one.
Acetaminophen made for infants is available in two different dose concentrations, and each concentration comes with its own medicine dropper or oral syringe. These dosing devices are not equal between the different concentrations. Using the wrong device may cause you to give your child an overdose of acetaminophen. Never mix and match dosing devices between infant formulations of acetaminophen.
You may need to shake the liquid before each use. Follow the directions on the medicine label.
The chewable tablet must be chewed thoroughly before you swallow it.
Make sure your hands are dry when handling the acetaminophen disintegrating tablet. Place the tablet on your tongue. It will begin to dissolve right away. Do not swallow the tablet whole. Allow it to dissolve in your mouth without chewing.
To use the acetaminophen effervescent granules, dissolve one packet of the granules in at least 4 ounces of water. Stir this mixture and drink all of it right away. To make sure you get the entire dose, add a little more water to the same glass, swirl gently and drink right away.
The oral powder should be placed directly on the tongue and swallowed.
Stop taking acetaminophen and call your doctor if:
you still have a sore throat after 2 days of use;
you still have a fever after 3 days of use;
you still have pain after 7 days of use (or 5 days if treating a child);
you have a skin rash, ongoing headache, nausea, vomiting, or any redness or swelling; or
if your symptoms get worse, or if you have any new symptoms.
This medication can cause unusual results with certain lab tests for glucose (sugar) in the urine. Tell any doctor who treats you that you are using acetaminophen.
Store at room temperature away from heat and moisture.
Detailed Acetaminophen dosage information
What happens if I miss a dose?
Since acetaminophen is taken as needed, you may not be on a dosing schedule. If you are taking the medication regularly, take the missed dose as soon as you remember. Skip the missed dose if it is almost time for your next scheduled dose. Do not take extra medicine to make up the missed dose.
What happens if I overdose?
Seek emergency medical attention or call the Poison Help line at 1-800-222-1222. An overdose of acetaminophen can be fatal.
The first signs of an acetaminophen overdose include loss of appetite, nausea, vomiting, stomach pain, sweating, and confusion or weakness. Later symptoms may include pain in your upper stomach, dark urine, and yellowing of your skin or the whites of your eyes."""

In [ ]:
doc = nlp(reply)
labels = [str(sentence) for sentence in list(doc.sents)]
examples = [question]

# Right answer

Do not take more than 5 doses of acetaminophen in 24 hours. 

Use only the number of milligrams per dose that is recommended for the child's weight and age. 

Use exactly as directed on the label

If you are treating a child, use a pediatric form of acetaminophen. 

Use only the special dose-measuring dropper or oral syringe that comes with the specific pediatric form you are using. 

Carefully follow the dosing directions on the medicine label.


In [ ]:
model_name = './'
model = SentenceSimilarity(model_name)
model.compute_label_embedding(labels)

Some weights of the model checkpoint at ./ were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ./ and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model 

In [ ]:
s.reset_index().sort_values(by='Similarity', ascending=False).reset_index().drop('index', axis=1)

,Sentences,Similarity
0,"If you are treating a child, use a pediatric f...",0.747708
1,Use acetaminophen exactly as directed on the l...,0.706305
2,Acetaminophen made for infants is available in...,0.703297
3,To use the acetaminophen effervescent granules...,0.696789
4,Using the wrong device may cause you to give y...,0.681803
5,Detailed Acetaminophen dosage information\n,0.675276
6,Stop taking acetaminophen and call your doctor...,0.671370
7,An overdose of acetaminophen can be fatal.\n,0.663842
8,Make sure your hands are dry when handling the...,0.658170
9,Use only the number of milligrams per dose tha...,0.656356


In [ ]:
scores = model.similarity(examples[0])
dic = {}

print(question)

for i, ex in enumerate(labels):
    dic[ex] = scores[i]

s = pd.Series(dic, name='Similarity')
s.index.name = 'Sentences'
df_finetune_bert = s.reset_index()
df_finetune_bert.sort_values(by='Similarity', ascending=False, inplace=True)
df_finetune_bert.reset_index(inplace=True)
df_finetune_bert.drop('index', axis=1, inplace=True)
df_finetune_bert

What is the uses of Gemfibrozil?


,Sentences,Similarity
0,"If you are treating a child, use a pediatric f...",0.747708
1,Use acetaminophen exactly as directed on the l...,0.706305
2,Acetaminophen made for infants is available in...,0.703297
3,To use the acetaminophen effervescent granules...,0.696789
4,Using the wrong device may cause you to give y...,0.681803
5,Detailed Acetaminophen dosage information\n,0.675276
6,Stop taking acetaminophen and call your doctor...,0.671370
7,An overdose of acetaminophen can be fatal.\n,0.663842
8,Make sure your hands are dry when handling the...,0.658170
9,Use only the number of milligrams per dose tha...,0.656356


In [ ]:
print(df_finetune_bert[df_finetune_bert['Sentences'].str.contains("Do not take more than 5 doses of acetaminophen in 24 hours.")], 
      df_finetune_bert[df_finetune_bert['Sentences'].str.contains("Use only the number of milligrams per dose that is recommended for the child's weight and age.")],
      df_finetune_bert[df_finetune_bert['Sentences'].str.contains("Use exactly as directed on the label")],
      df_finetune_bert[df_finetune_bert['Sentences'].str.contains("If you are treating a child, use a pediatric form of acetaminophen.")],
      df_finetune_bert[df_finetune_bert['Sentences'].str.contains("Use only the special dose-measuring dropper or oral syringe that comes with the specific pediatric form you are using.")],
      df_finetune_bert[df_finetune_bert['Sentences'].str.contains("Carefully follow the dosing directions on the medicine label.")],)

                                            Sentences  Similarity
20  Children younger than 12 years old: Do not tak...    0.614384                                            Sentences  Similarity
9  Use only the number of milligrams per dose tha...    0.656356                                   Sentences  Similarity
19  Use exactly as directed on the label.\n    0.618332                                            Sentences  Similarity
0  If you are treating a child, use a pediatric f...    0.747708                                             Sentences  Similarity
11  Use only the special dose-measuring dropper or...    0.645513                                             Sentences  Similarity
13  Carefully follow the dosing directions on the ...    0.632597


In [ ]:
model_name_bert = 'bert-large-uncased'
model_bert = SentenceSimilarity(model_name_bert)
model_bert.compute_label_embedding(labels)

scores = model_bert.similarity(examples[0])
dic = {}

print(question)

for i, ex in enumerate(labels):
    dic[ex] = scores[i]

s = pd.Series(dic, name='Similarity')
s.index.name = 'Sentences'
df_bert = s.reset_index()
df_bert.sort_values(by='Similarity', ascending=False, inplace=True)
df_bert.reset_index(inplace=True)
df_bert.drop('index', axis=1, inplace=True)
df_bert

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


What is the uses of Gemfibrozil?


,Sentences,Similarity
0,Detailed Acetaminophen dosage information\n,0.789242
1,"Since acetaminophen is taken as needed, you ma...",0.766882
2,An overdose of acetaminophen can be fatal.\n,0.765281
3,Avoid also using other medicines that contain ...,0.763936
4,"If you are treating a child, use a pediatric f...",0.759143
5,Stop taking acetaminophen and call your doctor...,0.756515
6,Never mix and match dosing devices between inf...,0.751906
7,An overdose of acetaminophen can damage your l...,0.744130
8,Acetaminophen made for infants is available in...,0.741706
9,This medication can cause unusual results with...,0.739916


In [ ]:
print(df_bert[df_bert['Sentences'].str.contains("Do not take more than 5 doses of acetaminophen in 24 hours.")], 
      df_bert[df_bert['Sentences'].str.contains("Use only the number of milligrams per dose that is recommended for the child's weight and age.")],
      df_bert[df_bert['Sentences'].str.contains("Use exactly as directed on the label")],
      df_bert[df_bert['Sentences'].str.contains("If you are treating a child, use a pediatric form of acetaminophen.")],
      df_bert[df_bert['Sentences'].str.contains("Use only the special dose-measuring dropper or oral syringe that comes with the specific pediatric form you are using.")],
      df_bert[df_bert['Sentences'].str.contains("Carefully follow the dosing directions on the medicine label.")],)

                                            Sentences  Similarity
14  Children younger than 12 years old: Do not tak...    0.709048                                             Sentences  Similarity
22  Use only the number of milligrams per dose tha...    0.685338                                   Sentences  Similarity
47  Use exactly as directed on the label.\n    0.466816                                            Sentences  Similarity
4  If you are treating a child, use a pediatric f...    0.759143                                             Sentences  Similarity
28  Use only the special dose-measuring dropper or...    0.670978                                             Sentences  Similarity
25  Carefully follow the dosing directions on the ...    0.677499


In [ ]:
model_name_xlm = 'vicgalle/xlm-roberta-large-xnli-anli'
model_xlm = SentenceSimilarity(model_name_xlm)
model_xlm.compute_label_embedding(labels)

scores = model_xlm.similarity(examples[0])
dic = {}

print(question)

for i, ex in enumerate(labels):
    dic[ex] = scores[i]

s = pd.Series(dic, name='Similarity')
s.index.name = 'Sentences'
df_xlm = s.reset_index()
df_xlm.sort_values(by='Similarity', ascending=False, inplace=True)
df_xlm

What is the uses of Gemfibrozil?


,Sentences,Similarity
38,Detailed Acetaminophen dosage information\n,0.991904
19,Never mix and match dosing devices between inf...,0.989643
14,liquid medicinewith the dosing syringe provide...,0.989598
45,Seek emergency medical attention or call the P...,0.986279
42,Skip the missed dose if it is almost time for ...,0.986243
28,To use the acetaminophen effervescent granules...,0.985514
22,The chewable tablet must be chewed thoroughly ...,0.984686
34,"you have a skin rash, ongoing headache, nausea...",0.984482
7,Use only the number of milligrams per dose tha...,0.984069
30,"To make sure you get the entire dose, add a li...",0.983659


In [ ]:
print(df_xlm[df_xlm['Sentences'].str.contains("Do not take more than 5 doses of acetaminophen in 24 hours.")], 
      df_xlm[df_xlm['Sentences'].str.contains("Use only the number of milligrams per dose that is recommended for the child's weight and age.")],
      df_xlm[df_xlm['Sentences'].str.contains("Use exactly as directed on the label")],
      df_xlm[df_xlm['Sentences'].str.contains("If you are treating a child, use a pediatric form of acetaminophen.")],
      df_xlm[df_xlm['Sentences'].str.contains("Use only the special dose-measuring dropper or oral syringe that comes with the specific pediatric form you are using.")],
      df_xlm[df_xlm['Sentences'].str.contains("Carefully follow the dosing directions on the medicine label.")],)

                                           Sentences  Similarity
6  Children younger than 12 years old: Do not tak...    0.970931                                            Sentences  Similarity
7  Use only the number of milligrams per dose tha...    0.984069                                  Sentences  Similarity
8  Use exactly as directed on the label.\n    0.955541                                             Sentences  Similarity
10  If you are treating a child, use a pediatric f...    0.978841                                             Sentences  Similarity
11  Use only the special dose-measuring dropper or...    0.982866                                             Sentences  Similarity
12  Carefully follow the dosing directions on the ...    0.957755
